In [1]:
%load_ext autoreload
%autoreload 2

In [350]:
import sys, os, pickle, time, warnings

import numpy as np, pandas as pd, scipy, scipy.stats as stats, tqdm, h5py
from copy import deepcopy as copy

sys.path.extend(['../utilities/', '../models/'])
import samplers, disk_cone_plcut as dcp, plotting, transformations
import disk_halo_mstogap as dh_msto
from transformations import func_inv_jac

from multiprocessing import Pool

# Load in parameter transformations

In [326]:
param_trans = {}
param_trans['shd'] = {'alpha1':('nexp',0,0,-3,3),
                      'alpha2':('nexp',0,0,-3,3)}
param_trans[0] = {'w':('exp',0,0,-10,10),
                  'fD': ('logit_scaled', 0,1, -10,10),
                  'alpha3':('nexp',0,0,-10,10),
                  'hz': ('logit_scaled', 0,  1.2,-10,10)}
param_trans[1] = {'w':('exp',0,0,-10,10),
                  'fD': ('logit_scaled', 0,1,-10,10),
                  'alpha3':('nexp',0,0,-10,10),
                  'hz': ('logit_scaled', 1.2,3,-10,10)}
param_trans[2] = {'w':('exp',0,0,-10,10),
                  'fD': ('logit_scaled', 0,1,-10,10),
                  'alpha3':('nexp',0,0,-10,10),
                  'hz': ('logit_scaled', 3,  7.3,-10,10)}

In [113]:
param_trans['shd'] = {'alpha1':('none',0,0,-3,3),
                      'alpha2':('none',0,0,-3,3)}

# Load data

In [327]:
# Load Sample
size = 1000
sample = {}; true_pars={}; latent_pars={};
filename = '/data/asfe2/Projects/mwtrace_data/mockmodel/sample.h'
with h5py.File(filename, 'r') as hf:
    subset = (hf['sample']['m'][...]>0)&(hf['sample']['m'][...]<33)
    print('%d/%d' % (np.sum(subset), len(subset)))
    subsample  = np.sort(np.random.choice(np.arange(np.sum(subset)), size=size, replace=False))
    for key in hf['sample'].keys():
        sample[key]=hf['sample'][key][...][subset][subsample]
    # Get true parameters
    for key in hf['true_pars'].keys():
        if not key in np.arange(3).astype(str):
            true_pars[key]=hf['true_pars'][key][...]
        else:
            true_pars[key]={}
            for par in hf['true_pars'][key].keys():
                true_pars[key][par]=hf['true_pars'][key][par][...]
for j in range(3): true_pars[str(j)]['w']*=size

99911/100000


# Generate model parameters

In [328]:
fid_pars = {'Mmax':true_pars['Mx'],  'lat_min':np.deg2rad(true_pars['theta_deg']), 'R0':true_pars['R0'],
            'free_pars':{}, 'fixed_pars':{}, 'functions':{}, 'functions_inv':{}, 'jacobians':{}, 'w':True,
            'components':['disk','disk','halo'], 'ncomponents':3}

fid_pars['free_pars'][0] = ['w', 'hz', 'fD']
fid_pars['free_pars'][1] = ['w', 'hz', 'fD']
fid_pars['free_pars'][2] = ['w', 'hz', 'fD']
fid_pars['free_pars']['shd'] = ['alpha1', 'alpha2']
ndim=np.sum([len(fid_pars['free_pars'][key]) for key in fid_pars['free_pars'].keys()])

fid_pars['fixed_pars'][0] = {'Mms':true_pars['Mms'], 'fD':1.-1e-15, 'alpha3':true_pars['0']['alpha3'],
                             'Mms1':true_pars['Mms1'], 'Mms2':true_pars['Mms2'],
                             'Mto':true_pars['0']['Mto']}
fid_pars['fixed_pars'][1] = copy(fid_pars['fixed_pars'][0]); fid_pars['fixed_pars'][2] = copy(fid_pars['fixed_pars'][0])
fid_pars['fixed_pars'][1]['Mto'] = true_pars['1']['Mto']
fid_pars['fixed_pars'][2]['Mto'] = true_pars['2']['Mto']

fid_pars['functions']={}; fid_pars['functions_inv']={}; fid_pars['jacobians']={}; bounds=[]
params_i = 0
for cmpt in np.arange(fid_pars['ncomponents']).tolist()+['shd',]:
    fid_pars['functions'][cmpt]={}; fid_pars['functions_inv'][cmpt]={}; fid_pars['jacobians'][cmpt]={}
    for par in fid_pars['free_pars'][cmpt]:
        fid_pars['functions'][cmpt][par], \
        fid_pars['functions_inv'][cmpt][par], \
        fid_pars['jacobians'][cmpt][par]=func_inv_jac[param_trans[cmpt][par][0]](*param_trans[cmpt][par][1:3])
        bounds.append([param_trans[cmpt][par][3], param_trans[cmpt][par][4]])
        params_i += 1;
bounds = np.array(bounds).T

In [329]:
true_pars

{'0': {'Mto': array(4.8),
  'alpha3': array(-1.),
  'fD': array(0.94),
  'hz': array(0.9),
  'w': array(200.)},
 '1': {'Mto': array(3.14),
  'alpha3': array(-0.5),
  'fD': array(0.998),
  'hz': array(1.9),
  'w': array(300.)},
 '2': {'Mto': array(3.3),
  'alpha3': array(-0.6),
  'fD': array(0.995),
  'hz': array(4.6),
  'w': array(500.)},
 'Mms': array(8.),
 'Mms1': array(9.),
 'Mms2': array(7.),
 'Mx': array(10.7),
 'N': array(100000),
 'R0': array(8.27),
 'alpha1': array(-0.15),
 'alpha2': array(-0.3),
 'theta_deg': array(60)}

In [330]:
true_params_f=[];
true_params=[]; 
for j in range(fid_pars['ncomponents']):
    for par in fid_pars['free_pars'][j]:
        true_params_f   += [fid_pars['functions_inv'][j][par](true_pars[str(j)][par]),]
        true_params += [true_pars[str(j)][par],]
for par in fid_pars['free_pars']['shd']:
    true_params_f += [fid_pars['functions_inv']['shd'][par](true_pars[par]),]
    true_params += [true_pars[par],]
true_params_f=np.array(true_params_f)
true_params=np.array(true_params)
true_params, true_params_f

(array([ 2.00e+02,  9.00e-01,  9.40e-01,  3.00e+02,  1.90e+00,  9.98e-01,
         5.00e+02,  4.60e+00,  9.95e-01, -1.50e-01, -3.00e-01]),
 array([ 5.29831737,  1.09861229,  2.75153531,  5.70378247, -0.45198512,
         6.2126061 ,  6.2146081 , -0.52324814,  5.29330482, -1.89711998,
        -1.2039728 ]))

# Test model

In [283]:
p0 = np.array( [transformations.logit(np.random.rand()),transformations.logit(np.random.rand()),-np.random.rand()*1,
                transformations.logit(np.random.rand()),transformations.logit(np.random.rand()),-np.random.rand()*1,
                transformations.logit(np.random.rand()),transformations.logit(np.random.rand()),-np.random.rand()*1,
                -np.random.rand()*1,-np.random.rand()*1] )

In [284]:
p0 = transformations.logit(np.random.rand(ndim))

In [285]:
p0 = np.array([0.,0.,-1.,
               0.,0.,-1.,
               0.,0.,-1.,
               -1.,-1.])

In [352]:
sample_2d = np.vstack((sample['parallax_obs'], sample['parallax_error'],
                       np.abs(sample['sinb']), np.log(np.sqrt(1-sample['sinb']**2)),
                       sample['m'], np.log(sample['parallax_error'])))
fid_pars['models']=[dh_msto.log_expmodel_perr_grad, dh_msto.log_expmodel_perr_grad, dh_msto.log_halomodel_perr_grad]
poisson_kwargs_global = {'sample':sample_2d, 'logmodel': dh_msto.logmodel_perr_grad, 
                         'model_integrate':dh_msto.integral_model,
                         'param_bounds':bounds, 'gmm':None, 'bins':([0,np.inf],[-np.inf,np.inf]),
                         'fid_pars':fid_pars, 'model_prior':None,
                        'chunksize':100, 'ncores':2}

In [332]:
def poisson_like(params, bounds=None, grad=False):

    poisson_kwargs = copy(poisson_kwargs_global)

    # Prior boundaries
    if bounds is None: bounds = poisson_kwargs['param_bounds']
    if np.sum((params<=bounds[0])|(params>=bounds[1]))>0:
        if grad: return -1e20, np.zeros(len(params))
        else: return -1e20

    # Optional prior inclusion
    if poisson_kwargs_global['model_prior'] is not None:
        prior=poisson_kwargs_global['model_prior'](params, fid_pars=poisson_kwargs['fid_pars'], grad=grad)
    else: 
        if not grad: prior=0.
        else: prior=(0.,0.)

    integral = poisson_kwargs['model_integrate'](params, bins=poisson_kwargs['bins'], fid_pars=poisson_kwargs['fid_pars'], grad=grad)
    obj = poisson_kwargs['logmodel'](poisson_kwargs['sample'], params, gmm=poisson_kwargs['gmm'], fid_pars=poisson_kwargs['fid_pars'], grad=grad)
    if not grad: return np.sum(obj) - integral + prior

    model_val = np.sum(obj[0]) - integral[0] + prior[0]
    model_grad = np.sum(obj[1], axis=1) - integral[1] + prior[1]
    return model_val, model_grad

In [384]:
global params_i

def poisson_like_parallel_func(ii):
    grad = poisson_kwargs_global['grad']
    output = poisson_kwargs_global['logmodel'](poisson_kwargs_global['sample'][:,ii:ii+poisson_kwargs_global['chunksize']].copy(),
                                            params_i,
                                            gmm=copy(poisson_kwargs_global['gmm']),
                                            fid_pars=copy(poisson_kwargs_global['fid_pars']),
                                            grad=grad)
    if not grad: return np.sum(obj)
    else: return np.sum(output[0]), np.sum(output[1], axis=1)
    
def poisson_like_parallel(params, bounds=None):
    
    global params_i
    params_i = params.copy()
    grad = poisson_kwargs_global['grad']
    
    poisson_kwargs = copy(poisson_kwargs_global)

    # Prior boundaries
    if bounds is None: bounds = poisson_kwargs['param_bounds']
    if np.sum((params<=bounds[0])|(params>=bounds[1]))>0:
        if grad: return -1e20, np.zeros(len(params))
        else: return -1e20

    # Optional prior inclusion
    if poisson_kwargs_global['model_prior'] is not None:
        prior=poisson_kwargs_global['model_prior'](params, fid_pars=poisson_kwargs['fid_pars'], grad=grad)
    else: 
        if not grad: prior=0.
        else: prior=(0.,0.)

    logl_val = 0.
    if grad: logl_grad = np.zeros(len(params))
            
    integral = poisson_kwargs['model_integrate'](params, bins=poisson_kwargs['bins'], 
                                                 fid_pars=poisson_kwargs['fid_pars'], grad=grad)
    
    i_list = np.arange(0, poisson_kwargs_global['sample'].shape[1], poisson_kwargs_global['chunksize'])
    with Pool(poisson_kwargs['ncores']) as p:
        for logl_i in p.imap(poisson_like_parallel_func, i_list):
            if not grad: logl_val += logl_i
            if grad: 
                logl_val += logl_i[0]
                logl_grad += logl_i[1]
    
    if not grad: return logl_val - integral[0] + prior[0]
    return logl_val - integral[0] + prior[0], logl_grad - integral[1] + prior[1]

In [386]:
poisson_kwargs_global['grad']=True

In [387]:
poisson_like_parallel(p0)

(-1147.2148089940306,
 array([ 164.10198065,  188.44085838,   79.91249312,  151.75721547,
         -13.7860862 ,   60.30290338,  679.32722189,  221.0192986 ,
         -68.61722509, -237.45785972,  -28.72894533]))

In [381]:
poisson_like(p0, grad=True)

(-1147.2148089940306,
 array([ 164.10198065,  188.44085838,   79.91249312,  151.75721547,
         -13.7860862 ,   60.30290338,  679.32722189,  221.0192986 ,
         -68.61722509, -237.45785972,  -28.72894533]))

In [204]:
model = lambda x: poisson_like(x, grad=False)
grad  = lambda x: poisson_like(x, grad=True)[1]

In [205]:
np.vstack((scipy.optimize.approx_fprime(p0, model, 1e-10), grad(p0))).T

array([[ 2.70630380e+02,  2.70629917e+02],
       [ 4.59522198e+01,  4.59477087e+01],
       [ 0.00000000e+00, -1.03400554e-13],
       [ 3.41143505e+02,  3.41143153e+02],
       [ 3.17072590e+00,  3.19474612e+00],
       [ 0.00000000e+00, -2.82754267e-13],
       [ 3.85226713e+02,  3.85226929e+02],
       [-5.21561105e+01, -5.24186719e+01],
       [ 0.00000000e+00, -2.61137875e-13],
       [-9.33573574e+01, -9.41802020e+01],
       [ 2.73075784e+00,  3.12589444e+00]])

In [388]:
model = lambda x: poisson_like_parallel(x)[0]
grad  = lambda x: poisson_like_parallel(x)[1]

In [389]:
np.vstack((scipy.optimize.approx_fprime(p0, model, 1e-10), grad(p0))).T

array([[ 164.10012904,  164.10198065],
       [ 188.44730221,  188.44085838],
       [  79.91502571,   79.91249312],
       [ 151.7560122 ,  151.75721547],
       [ -13.78339221,  -13.7860862 ],
       [  60.29949873,   60.30290338],
       [ 679.32660386,  679.32722189],
       [ 221.36191546,  221.0192986 ],
       [ -68.61455404,  -68.61722509],
       [-234.34949981, -237.45785972],
       [ -28.50811143,  -28.72894533]])

# Run Newton CG

In [335]:
from scipy.optimize import minimize

In [341]:
##Print callback function
def printx(Xi):
    global Nfeval
    global fout
    sys.stdout.write('At iterate {0}, {1}'.format(Nfeval, poisson_like(Xi)) + '\n')
    sys.stdout.write(str(Xi)+'\n')
    Nfeval += 1

In [337]:
def nll(x):
    lnl, grad = poisson_like(x, grad=True)
    return -lnl, -grad

In [338]:
p0 = np.array( [transformations.logit(np.random.rand()),transformations.logit(np.random.rand()),-np.random.rand()*1,
                transformations.logit(np.random.rand()),transformations.logit(np.random.rand()),-np.random.rand()*1,
                transformations.logit(np.random.rand()),transformations.logit(np.random.rand()),-np.random.rand()*1,
                -np.random.rand()*1,-np.random.rand()*1] )

p0 = np.array([0.,0.,-1.,0.,
               0.,0.,-1.,0.,
               0.,0.,-1.,0.,
               -1.,-1.])

p0 = transformations.logit(np.random.rand(ndim))
p0 = np.random.normal(0,1,ndim)

In [339]:
nll(p0)

(1147.2148089940306,
 array([-164.10198065, -188.44085838,  -79.91249312, -151.75721547,
          13.7860862 ,  -60.30290338, -679.32722189, -221.0192986 ,
          68.61722509,  237.45785972,   28.72894533]))

In [342]:
Nfeval=1
res = minimize(nll, p0, method='Newton-CG', jac=True, callback=printx, options={'disp': True})

At iterate 1, 3598.895787182455
[ 1.63624877  0.75507424  0.57109867  0.67120343 -0.09559155  0.60756163
  7.2821046   0.61782692  0.83389269 -1.81231508  0.10851004]At iterate 2, 3714.9285581788154
[ 1.63751846  0.75432765  0.57207958  0.67070505 -0.09568277  0.60789415
  6.90704201  0.58606947  0.99814073 -1.83250911  0.09245845]At iterate 3, 3843.883564935802
[ 1.66881821  0.74444813  0.58676435  0.66983404 -0.09693978  0.61232193
  6.93242751  0.37544015  2.13933082 -2.06682868 -0.09940385]At iterate 4, 3873.5971584437852
[ 1.72136958  0.72498583  0.61295375  0.66727917 -0.09916159  0.61885818
  6.90039239  0.39242968  2.88821645 -2.41640392 -0.47017023]At iterate 5, 3889.079356798249
[ 1.88374089  0.65581969  0.69761948  0.66195473 -0.10650763  0.63405112
  6.89579388  0.45232086  3.79593015 -2.17522113 -0.84492366]At iterate 6, 3912.245993384174
[ 3.87690995 -0.17398531  1.48781876  0.69598885 -0.19165654  0.69950227
  6.87008117  0.51383388  6.00449536 -2.23624388 -1.45646684]At

  6.44276908 -0.21792048  6.57931316 -2.43831362 -2.3077599 ]At iterate 51, 3957.990679445248
[ 5.90556104  4.62063125  3.80357347  1.32325    -0.75720326  0.3407291
  6.44446699 -0.21805416  6.57955907 -2.43826639 -2.30793893]At iterate 52, 3957.9919812221333
[ 5.90479396  4.62173677  3.80441993  1.32546681 -0.75890023  0.33944386
  6.44275728 -0.21792966  6.58018612 -2.43857717 -2.30812224]At iterate 53, 3957.9931779732497
[ 5.90559813  4.62221018  3.80478844  1.32642475 -0.75962496  0.33889612
  6.44462491 -0.21807665  6.58045449 -2.4385246  -2.30831821]At iterate 54, 3957.994219134876
[ 5.90476651  4.62311362  3.80548612  1.32824425 -0.76101948  0.33784064
  6.44275787 -0.21793868  6.58096803 -2.43881245 -2.30844708]At iterate 55, 3957.995405858067
[ 5.90550201  4.62354682  3.8058261   1.32912457 -0.76168556  0.33733758
  6.44446648 -0.21807316  6.58121403 -2.43876449 -2.30862652]At iterate 56, 3957.996721638865
[ 5.90472981  4.62464964  3.8066854   1.33135601 -0.7633941   0.336045

  6.44448538 -0.21826466  6.59955124 -2.44429528 -2.31619122]At iterate 100, 3958.0524748981707
[ 5.903933    4.65620439  3.83392696  1.39864057 -0.81484501  0.29752735
  6.4426459  -0.21812856  6.60018754 -2.44461929 -2.31636849]At iterate 101, 3958.0538393129714
[ 5.90478994  4.65665659  3.83435871  1.39965708 -0.81561744  0.29695581
  6.44464334 -0.21828521  6.6004558  -2.44455774 -2.31656919]At iterate 102, 3958.055036807622
[ 5.90389785  4.6575435   3.83520052  1.40164067 -0.81714572  0.29582316
  6.44264911 -0.21813612  6.60098356 -2.44485999 -2.31669483]At iterate 103, 3958.0563912895814
[ 5.90468712  4.65796037  3.83560168  1.40258147 -0.81786087  0.29529453
  6.44448934 -0.21828046  6.60123133 -2.44480347 -2.31687997]At iterate 104, 3958.057884952552
[ 5.90385197  4.65902938  3.83662494  1.40498279 -0.8197096   0.29392589
  6.44263862 -0.21814335  6.60186849 -2.44512868 -2.3170568 ]At iterate 105, 3958.0592643418836
[ 5.90471343  4.65947988  3.8370623   1.40600399 -0.8204861  

  6.44258311 -0.21828809  6.62053348 -2.45080694 -2.32466166]At iterate 149, 3958.1229258522603
[ 5.90377544  4.69007892  3.86946342  1.47820412 -0.87617319  0.25271749
  6.44471374 -0.21845477  6.62080306 -2.45073678 -2.32486835]At iterate 150, 3958.1242939935373
[ 5.90282159  4.69094937  3.87047123  1.48034384 -0.87784224  0.25151637
  6.44258931 -0.21829419  6.62134716 -2.45105493 -2.32499217]At iterate 151, 3958.1258311537313
[ 5.90366521  4.69135084  3.87094146  1.48133971 -0.87860814  0.250967
  6.44456363 -0.21844866  6.62159776 -2.45099018 -2.32518401]At iterate 152, 3958.127518291543
[ 5.90276627  4.69238861  3.87215261  1.48390057 -0.88060491  0.24953223
  6.4425804  -0.21829963  6.62224759 -2.45133064 -2.32535706]At iterate 153, 3958.1290813247597
[ 5.90368134  4.69281983  3.87266196  1.48497437 -0.88143129  0.2489404
  6.44472195 -0.21846714  6.62251736 -2.45125981 -2.32556429]At iterate 154, 3958.1304640369153
[ 5.90272247  4.69368879  3.87368444  1.48712588 -0.883112    0

  6.44483297 -0.21858559  6.64159384 -2.45710845 -2.33328354]At iterate 198, 3958.202350229732
[ 5.90153769  4.72338774  3.91201635  1.56367741 -0.94323352  0.20544013
  6.44258675 -0.21841412  6.64215643 -2.45744299 -2.33340736]At iterate 199, 3958.20406805218
[ 5.90243166  4.72377474  3.91256255  1.56471508 -0.94404961  0.2048789
  6.4446868  -0.21857818  6.64241106 -2.45737103 -2.33360645]At iterate 200, 3958.2059443622975
[ 5.90147321  4.72478306  3.91398278  1.56740233 -0.94619507  0.20340041
  6.44257942 -0.21841786  6.64307683 -2.4577276  -2.33377816]At iterate 201, 3958.207688484694
[ 5.90243758  4.72519653  3.91457093  1.56851421 -0.94707059  0.20279962
  6.44484485 -0.21859481  6.64334951 -2.4576494  -2.33399211]At iterate 202, 3958.2092399625813
[ 5.90142174  4.72604887  3.91578082  1.57079124 -0.948893    0.20154628
  6.44258928 -0.21842249  6.64391379 -2.45798531 -2.33411606]At iterate 203, 3958.2109719019863
[ 5.90231961  4.72643471  3.91633357  1.57183148 -0.94971315  0.

  6.44264944 -0.2185      6.6635071  -2.46406141 -2.34200686]At iterate 247, 3958.2903642486954
[ 5.90100042  4.75527857  3.96099088  1.65120823 -1.01404877  0.15796181
  6.44485005 -0.21867185  6.6637667  -2.46398434 -2.34221287]At iterate 248, 3958.2923986539195
[ 5.89999517  4.75625977  3.96263294  1.65396187 -1.01633578  0.15647476
  6.4426446  -0.21850235  6.66445193 -2.46435644 -2.3423867 ]At iterate 249, 3958.2942923294777
[ 5.90099761  4.75665703  3.96330286  1.65508679 -1.0172565   0.15587937
  6.44500799 -0.21868689  6.66472896 -2.46427315 -2.34260724]At iterate 250, 3958.2959822939297
[ 5.89993743  4.75749169  3.96470953  1.65743158 -1.01921041  0.15461247
  6.44265792 -0.21850584  6.66531347 -2.46462427 -2.34273389]At iterate 251, 3958.2978610489167
[ 5.90087335  4.75786379  3.965341    1.65848669 -1.02007574  0.15405445
  6.44486518 -0.21867821  6.66557354 -2.46454691 -2.34294043]At iterate 252, 3958.2999056604326
[ 5.89986503  4.7588427   3.96700141  1.66124179 -1.0223735

  6.44504085 -0.21873577  6.68573677 -2.47085645 -2.35106543]At iterate 296, 3958.3845730847015
[ 5.89838337  4.78685252  4.01815547  1.74105913 -1.09051872  0.11014778
  6.44278176 -0.21856077  6.68644326 -2.47124158 -2.35124536]At iterate 297, 3958.3865494602055
[ 5.89940497  4.78723514  4.01890337  1.74216113 -1.09147683  0.10957647
  6.44519837 -0.21874961  6.68672578 -2.47115675 -2.35147146]At iterate 298, 3958.388314609458
[ 5.8983219   4.78805014  4.02049731  1.74448532 -1.09353842  0.10834311
  6.44279827 -0.21856345  6.6873303  -2.47151996 -2.35160379]At iterate 299, 3958.3902723524075
[ 5.89927667  4.78840926  4.021203    1.74551941 -1.09444021  0.10780738
  6.44505749 -0.21874001  6.68759608 -2.47144117 -2.35181589]At iterate 300, 3958.392398946412
[ 5.89824535  4.78936224  4.02307672  1.74823806 -1.096856    0.10636778
  6.44279644 -0.21856477  6.68830437 -2.47182723 -2.35199652]At iterate 301, 3958.394378367518
[ 5.8992675   4.78974371  4.02383067  1.74933624 -1.09781675  

In [295]:
Nfeval=1
res = minimize(nll, p0, method='Newton-CG', jac=True, callback=printx, options={'disp': True})

At iterate 1, 382.1533510967495
At iterate 2, 1417.8559603886442
At iterate 3, 3342.7086098593636
At iterate 4, 3856.3236068156734
At iterate 5, 3895.662692565562
At iterate 6, 3913.5752934549764
At iterate 7, 3921.107656502854
At iterate 8, 3921.880395782853
At iterate 9, 3923.3299392899216
At iterate 10, 3925.707531290581
At iterate 11, 3927.852789996343
At iterate 12, 3927.9902074533043
At iterate 13, 3928.2291579291873
At iterate 14, 3928.6011691844624
At iterate 15, 3928.9054650820626
At iterate 16, 3929.102004722321
At iterate 17, 3929.276938070457
At iterate 18, 3929.367395499419
At iterate 19, 3929.3703735462186
At iterate 20, 3929.3918575758535
At iterate 21, 3929.4245457653374
At iterate 22, 3929.465797099367
         Current function value: -3929.465797
         Iterations: 22
         Function evaluations: 37
         Gradient evaluations: 701
         Hessian evaluations: 0


In [343]:
res

     fun: -3959.5666706320394
     jac: array([-0.06705799, -0.03714148, -0.02470247,  0.03652415,  0.04918254,
        0.00235685,  0.03558   , -0.085792  , -0.03812292,  0.26117587,
       -0.18593687])
 message: 'Warning: Desired error not necessarily achieved due to precision loss.'
    nfev: 382
    nhev: 0
     nit: 322
    njev: 2146
  status: 2
 success: False
       x: array([ 5.88487021,  6.05180987,  5.87061082,  2.20342148, -3.88828526,
       -0.3914046 ,  6.44792849, -0.22244159,  7.9444138 , -2.96098234,
       -2.99394063])

In [344]:
true_params_f

array([ 5.29831737,  1.09861229,  2.75153531,  5.70378247, -0.45198512,
        6.2126061 ,  6.2146081 , -0.52324814,  5.29330482, -1.89711998,
       -1.2039728 ])

In [297]:
p0

array([-0.19084477, -0.56673929, -0.34582121,  0.97962138, -0.0138391 ,
        0.10512387,  1.4598857 , -1.34547288,  0.45794454, -1.05047223,
        0.28290758])

In [299]:
nll(p0)

(195.25450062310043,
 array([-155.90680127,  -33.4945419 , 1175.75934963, -442.40285048,
          37.3496572 , 7032.76624098, -393.89517243, -143.37709121,
        2885.68067811,  188.03504714,  168.47426356]))

In [305]:
test_params = true_params_f.copy()
test_params[0]=8

In [306]:
nll(test_params)

(-2192.6777290664677,
 array([ 2.41767138e+03, -9.69434500e+01,  1.25680079e+02,  1.92025531e+02,
        -1.84331470e+01,  6.13785206e-01,  1.71261076e+02,  6.25043693e+01,
        -5.43630866e-02, -8.12379636e+01,  2.92653763e+01]))

In [298]:
nll(true_params_f)

(-3926.2765947426215,
 array([-6.43000386,  3.43838773,  8.59572888, -1.56281194,  4.24950048,
         0.75684362,  7.9928158 ,  0.24131594,  1.05720863,  9.87266071,
         2.6715648 ]))

In [227]:
p0

array([ 0.,  0., -1.,  0.,  0., -1.,  0.,  0., -1., -1., -1.])

In [308]:
model = lambda x: nll(x)[0]
grad  = lambda x: nll(x)[1]
np.vstack((scipy.optimize.approx_fprime(p0, model, 1e-10), grad(p0))).T

array([[-155.90728708, -155.90680127],
       [ -33.49811095,  -33.4945419 ],
       [   0.        , 1175.75934963],
       [-442.40294983, -442.40285048],
       [  37.3412945 ,   37.3496572 ],
       [   0.        , 7032.76624098],
       [-393.89590256, -393.89517243],
       [-143.54554878, -143.37709121],
       [   0.        , 2885.68067811],
       [ 187.29792828,  188.03504714],
       [ 169.21745782,  168.47426356]])

In [315]:
fid_pars['functions'][0]['fD'](-0.3)

0.425557483188341

In [320]:
model(p0)

{0: {'alpha2': -1.3269825113701028, 'alpha1': -0.349772537632544, 'w': 0.8262608369484923, 'alpha3': array(-1.), 'Mto': array(4.8), 'Mms2': array(7.), 'Mms1': array(9.), 'fD': 0.4143961333462523, 'hz': 0.4343874678960608, 'Mms': array(8.)}, 1: {'alpha2': -1.3269825113701028, 'alpha1': -0.349772537632544, 'w': 2.663447610428108, 'alpha3': array(-1.), 'Mto': array(3.14), 'Mms2': array(7.), 'Mms1': array(9.), 'fD': 0.5262567914035446, 'hz': 2.09377250509069, 'Mms': array(8.)}, 2: {'alpha2': -1.3269825113701028, 'alpha1': -0.349772537632544, 'w': 4.305467371675508, 'alpha3': array(-1.), 'Mto': array(3.3), 'Mms2': array(7.), 'Mms1': array(9.), 'fD': 0.6125264518808964, 'hz': 3.8884293917911075, 'Mms': array(8.)}}


321.0295472116861

In [313]:
p0

array([-0.19084477, -0.56673929, -0.34582121,  0.97962138, -0.0138391 ,
        0.10512387,  1.4598857 , -1.34547288,  0.45794454, -1.05047223,
        0.28290758])

In [323]:
ptest = true_params_f
#ptest = p0
np.vstack((scipy.optimize.approx_fprime(ptest, model, 1e-10), grad(ptest))).T

array([[-6.29370334, -6.29036821],
       [ 2.17369234,  2.17617262],
       [-2.38287612, -2.37843458],
       [ 0.51841198,  0.52300828],
       [ 2.56477506,  2.57046684],
       [-0.16370905, -0.15342221],
       [ 5.75710146,  5.76735993],
       [ 0.41836756,  0.69758825],
       [-0.20008883, -0.20279447],
       [ 0.08185452,  0.72938671],
       [ 0.76397555,  0.52209811]])

In [307]:
Nfeval=1
res_hot = minimize(nll, true_params_f, method='Newton-CG', jac=True, callback=printx, options={'disp': True})

         Current function value: -3926.276595
         Iterations: 0
         Function evaluations: 17
         Gradient evaluations: 16
         Hessian evaluations: 0


In [222]:
nll(p0)

(976.3072865801602,
 array([-2.70629917e+02, -4.59477087e+01,  1.03400554e-13, -3.41143153e+02,
        -3.19474612e+00,  2.82754267e-13, -3.85226929e+02,  5.24186719e+01,
         2.61137875e-13,  9.41802020e+01, -3.12589444e+00]))

In [223]:
nll(p0+0.01)

(976.3072865801602,
 array([-2.70629917e+02, -4.59477087e+01,  1.03400554e-13, -3.41143153e+02,
        -3.19474612e+00,  2.82754267e-13, -3.85226929e+02,  5.24186719e+01,
         2.61137875e-13,  9.41802020e+01, -3.12589444e+00]))

In [207]:
Nfeval = 1
with fout as open('Newton_cg_test.txt','w'):
    res = minimize(poisson_like, p0, method='NewtonCG', jac=True, callback=printx, options={'disp': True})

SyntaxError: can't assign to function call (<ipython-input-207-f839da3fc25c>, line 2)

In [149]:
isource=0
model = lambda x: poisson_kwargs_global['logmodel'](sample_2d, x, fid_pars=fid_pars, grad=False)[isource]
grad  = lambda x: poisson_kwargs_global['logmodel'](sample_2d, x, fid_pars=fid_pars, grad=True)[1][:,isource]

In [150]:
np.vstack((scipy.optimize.approx_fprime(p0, model, 1e-10), grad(p0))).T

[ 0.  0. -1.  0.  0. -1.  0.  0. -1. -1. -1.] [ 1.          0.3        -0.36787944  1.          0.45       -0.36787944
  1.          1.075      -0.36787944 -0.36787944 -0.36787944]


array([[ 1.42517109e-01,  1.42518274e-01],
       [ 2.54443133e-01,  2.54438904e-01],
       [ 0.00000000e+00,  3.43633871e-50],
       [ 5.52504709e-01,  5.52505878e-01],
       [-6.95665747e-02, -6.95668001e-02],
       [ 0.00000000e+00,  2.23578165e-36],
       [ 3.04973824e-01,  3.04975849e-01],
       [ 1.28146382e-01,  1.28119817e-01],
       [ 0.00000000e+00,  1.51371304e-19],
       [ 2.71687117e-01, -1.69670768e+03],
       [-4.10316225e-02, -6.92702658e+01]])

In [160]:
isource=0
model = lambda x: poisson_kwargs_global['logmodel'](sample_2d[:,isource:isource+1], x, fid_pars=fid_pars, grad=False)[0]
grad  = lambda x: poisson_kwargs_global['logmodel'](sample_2d[:,isource:isource+1], x, fid_pars=fid_pars, grad=True)[1][:,0]
model = lambda x: poisson_kwargs_global['logmodel'](sample_2d, x, fid_pars=fid_pars, grad=False)#[isource]
grad  = lambda x: poisson_kwargs_global['logmodel'](sample_2d, x, fid_pars=fid_pars, grad=True)[1]#[:,isource]

In [159]:
grad(p0)

[ 0.  0. -1.  0.  0. -1.  0.  0. -1. -1. -1.] [ 1.          0.3        -0.36787944  1.          0.45       -0.36787944
  1.          1.075      -0.36787944 -0.36787944 -0.36787944]


array([ 1.42518274e-01,  2.54438904e-01,  3.43633871e-50,  5.52505878e-01,
       -6.95668001e-02,  2.23578165e-36,  3.04975849e-01,  1.28119817e-01,
        1.51371304e-19,  2.71272528e-01, -4.09447053e-02])

In [164]:
grad(p0)[:,0]

[ 0.  0. -1.  0.  0. -1.  0.  0. -1. -1. -1.] [ 1.          0.3        -0.36787944  1.          0.45       -0.36787944
  1.          1.075      -0.36787944 -0.36787944 -0.36787944]


array([ 1.42518274e-01,  2.54438904e-01,  3.43633871e-50,  5.52505878e-01,
       -6.95668001e-02,  2.23578165e-36,  3.04975849e-01,  1.28119817e-01,
        1.51371304e-19, -1.69670768e+03, -6.92702658e+01])

In [154]:
np.vstack((scipy.optimize.approx_fprime(p0, model, 1e-10), grad(p0))).T

[ 0.  0. -1.  0.  0. -1.  0.  0. -1. -1. -1.] [ 1.          0.3        -0.36787944  1.          0.45       -0.36787944
  1.          1.075      -0.36787944 -0.36787944 -0.36787944]


array([[ 1.42517109e-01,  1.42518274e-01],
       [ 2.54443133e-01,  2.54438904e-01],
       [ 0.00000000e+00,  3.43633871e-50],
       [ 5.52504709e-01,  5.52505878e-01],
       [-6.95665747e-02, -6.95668001e-02],
       [ 0.00000000e+00,  2.23578165e-36],
       [ 3.04973824e-01,  3.04975849e-01],
       [ 1.28146382e-01,  1.28119817e-01],
       [ 0.00000000e+00,  1.51371304e-19],
       [ 2.71687117e-01, -1.69670768e+03],
       [-4.10316225e-02, -6.92702658e+01]])

In [181]:
isource=0
model = lambda x: dh_msto.logmodel_perr_grad(sample_2d[:,isource:isource+1], x, fid_pars=fid_pars, grad=False)[0]
grad  = lambda x: dh_msto.logmodel_perr_grad(sample_2d[:,isource:isource+1], x, fid_pars=fid_pars, grad=True)[1][:,0]

In [183]:
np.vstack((scipy.optimize.approx_fprime(p0, model, 1e-10), grad(p0))).T

-6.554203076503464e-49 -1.2047312338610843
-1.0999855140655041e-35 -0.7742772605480664


array([[ 1.42517109e-01,  1.42518274e-01],
       [ 2.54443133e-01,  2.54438904e-01],
       [ 0.00000000e+00,  3.43633871e-50],
       [ 5.52504709e-01,  5.52505878e-01],
       [-6.95665747e-02, -6.95668001e-02],
       [ 0.00000000e+00,  2.23578165e-36],
       [ 3.04973824e-01,  3.04975849e-01],
       [ 1.28146382e-01,  1.28119817e-01],
       [ 0.00000000e+00,  1.51371304e-19],
       [ 2.71687117e-01,  2.71272528e-01],
       [-4.10316225e-02, -4.09447053e-02]])

In [184]:
isource=0
model = lambda x: dh_msto.logmodel_perr_grad(sample_2d, x, fid_pars=fid_pars, grad=False)[isource]
grad  = lambda x: dh_msto.logmodel_perr_grad(sample_2d, x, fid_pars=fid_pars, grad=True)[1][:,isource]

In [169]:
np.vstack((scipy.optimize.approx_fprime(p0, model, 1e-10), grad(p0))).T

(1000, 3)


array([[ 1.42517109e-01,  1.42518274e-01],
       [ 2.54443133e-01,  2.54438904e-01],
       [ 0.00000000e+00,  3.43633871e-50],
       [ 5.52504709e-01,  5.52505878e-01],
       [-6.95665747e-02, -6.95668001e-02],
       [ 0.00000000e+00,  2.23578165e-36],
       [ 3.04973824e-01,  3.04975849e-01],
       [ 1.28146382e-01,  1.28119817e-01],
       [ 0.00000000e+00,  1.51371304e-19],
       [ 2.71687117e-01, -1.69670768e+03],
       [-4.10316225e-02, -6.92702658e+01]])

In [189]:
np.vstack((scipy.optimize.approx_fprime(p0, model, 1e-10), grad(p0))).T

array([[ 1.42517109e-01,  1.42518274e-01],
       [ 2.54443133e-01,  2.54438904e-01],
       [ 0.00000000e+00,  3.43633871e-50],
       [ 5.52504709e-01,  5.52505878e-01],
       [-6.95665747e-02, -6.95668001e-02],
       [ 0.00000000e+00,  2.23578165e-36],
       [ 3.04973824e-01,  3.04975849e-01],
       [ 1.28146382e-01,  1.28119817e-01],
       [ 0.00000000e+00,  1.51371304e-19],
       [ 2.71687117e-01,  2.71272528e-01],
       [-4.10316225e-02, -4.09447053e-02]])

In [50]:
p0

array([ 0.84804686,  1.19282023, -2.4230274 , -1.63610134, -2.41229485,
        1.91579073, -0.67597475, -0.8093295 ])

In [65]:
fid_pars['free_pars']

{0: ['w', 'hz'], 1: ['w', 'hz'], 2: ['w', 'hz'], 'shd': ['alpha1', 'alpha2']}

In [104]:
test_arr = np.linspace(0,10,11)
fid_pars['jacobians']['shd']['alpha1'](test_arr)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [61]:
fid_pars['functions']['shd']['alpha1'](0.)

0.0

In [62]:
fid_pars['functions_inv']['shd']['alpha1'](-1.)

-1.0

In [42]:
param_trans['shd']['alpha1']

('nexp', 0, 0, -3, 3)

In [31]:
fid_pars['free_pars']['shd']

['alpha1', 'alpha2']

In [35]:
fid_pars['functions']['shd']

{'alpha1': <function transformations.<lambda>.<locals>.<lambda>(x)>,
 'alpha2': <function transformations.<lambda>.<locals>.<lambda>(x)>}

In [34]:
fid_pars['jacobians']['shd']

{'alpha1': <function transformations.<lambda>.<locals>.<lambda>(x)>,
 'alpha2': <function transformations.<lambda>.<locals>.<lambda>(x)>}

In [ ]:
if True:
    print('Full fit, no error:')
    sample_2d = np.vstack((1/sample['s'], np.log(1/sample['s']),
                             sample['sinb'], np.log(np.sqrt(1-sample['sinb']**2)),
                             sample['m']))
    fid_pars['models']=[dh_msto.log_expmodel_grad, dh_msto.log_expmodel_grad, dh_msto.log_halomodel_grad]
    poisson_kwargs_global = {'sample':sample_2d,
                             'logmodel': dh_msto.logmodel_grad, 'model_integrate':dh_msto.integral_model,
                             'param_bounds':bounds, 'gmm':None, 'bins':([0,np.inf],[-np.inf,np.inf]),
                             'fid_pars':fid_pars, 'model_prior':None}
    print('poisson_like(p0): %.2e' % poisson_like(p0))

    sampler = samplers.run_mcmc_global(p0, poisson_like, bounds, nstep=nsteps, ncores=ncores, notebook=False)
    output['chain']['full_noerr'] = sampler.chain
    output['lnprob']['full_noerr'] = sampler.lnprobability

#save_hdf5(output, savefile)s

if True:
    print('Full fil, parallax error:')
    sample_2d = np.vstack((sample['parallax_obs'], sample['parallax_error'],
                           np.abs(sample['sinb']), np.log(np.sqrt(1-sample['sinb']**2)),
                           sample['m'], np.log(sample['parallax_error'])))
    fid_pars['models']=[dh_msto.log_expmodel_perr, dh_msto.log_expmodel_perr, dh_msto.log_halomodel_perr]
    poisson_kwargs_global = {'sample':sample_2d, 'logmodel': dh_msto.logmodel_perr, 'model_integrate':dh_msto.integral_model,
                             'param_bounds':bounds, 'gmm':None, 'bins':([0,np.inf],[-np.inf,np.inf]),
                             'fid_pars':fid_pars, 'model_prior':None}
    print('poisson_like(p0): %.2e' % poisson_like(p0))

    sampler = samplers.run_mcmc_global(p0, poisson_like, bounds, nstep=nsteps, ncores=ncores, notebook=False)
    output['chain']['full_perr'] = sampler.chain
    output['lnprob']['full_perr'] = sampler.lnprobability

save_hdf5(output, savefile)